In [26]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

Most Active Stocks

In [32]:
for page in range(0, 201, 25):
    url = f'https://finance.yahoo.com/markets/stocks/most-active/?start=-{page}&count=25'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html')

    columns = soup.find_all('div', class_='header-container yf-1dbt8wv')
    header = [name.text.strip() for name in columns]
    header.insert(1, "Name")

    data = []
    body = soup.find('tbody')
    rows = body.find_all('tr', class_= 'yf-1dbt8wv')

    for row in rows:
        point = row.find_all('td', class_='cell yf-1dbt8wv') 
        line = [case.text.strip() for case in point] 
        splitter = line[0].split(" ", 1)
        line = splitter + line[1:]
        line[1] = line[1].strip()
        line[2] = line[2].split(" ", 1)[0]
        data.append(line)

In [31]:
url = 'https://finance.yahoo.com/markets/stocks/most-active/?start=0&count=203'
# time.sleep(5)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

columns = soup.find_all('div', class_='header-container yf-1dbt8wv')
header = [name.text.strip() for name in columns]
header.insert(1, "Name")

data = []
body = soup.find('tbody')
rows = body.find_all('tr', class_= 'yf-1dbt8wv')

for row in rows:
    point = row.find_all('td', class_='cell yf-1dbt8wv') 
    line = [case.text.strip() for case in point] 
    splitter = line[0].split(" ", 1)
    line = splitter + line[1:]
    line[1] = line[1].strip()
    line[2] = line[2].split(" ", 1)[0]
    data.append(line)

In [28]:
df = pd.DataFrame(data, columns=header)
df = df.drop(['52 Wk Range', 'Day Chart'], axis=1)
df.count()

Symbol             25
Name               25
Price              25
Change             25
Change %           25
Volume             25
Avg Vol (3M)       25
Market Cap         25
P/E Ratio (TTM)    25
52 Wk Change %     25
dtype: int64

In [43]:
url = 'https://finance.yahoo.com/markets/stocks/most-active/?start=0&count=203'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup.find_all('tr', id=24)

[<tr class="row false yf-1dbt8wv" id="24"><td class="cell yf-1dbt8wv"><span class="yf-1dbt8wv"><div style="display: contents; --background-color:transparent; --hover-bg-color:transparent; --hover-color:var(--enabled-active-emph); --text-color:var(--enabled-active-emph); --hover-focus-color:var(--hovered-emph-same); --text-decoration:underline;"><a aria-label="MicroStrategy Incorporated" class="ticker medium hover stacked yf-138ga19" data-testid="table-cell-ticker" data-ylk="elm:qte;elmt:link;itc:0;sec:stocks-datatable;slk:MSTR;subsec:most-active" href="/quote/MSTR/" target="_parent" title="MicroStrategy Incorporated"> <div class="name yf-138ga19 stacked"> <span class="symbol yf-138ga19">MSTR </span> <span class="yf-138ga19 longName" title="MicroStrategy Incorporated">MicroStrategy Incorporated</span></div> </a></div></span> </td><td class="cell yf-1dbt8wv"><span class="yf-1dbt8wv"> <fin-streamer active="" data-field="regularMarketPrice" data-pricehint="2" data-symbol="MSTR" data-test="

In [14]:
df

,Symbol,Name,Price,Change,Change %,Volume,Avg Vol (3M),Market Cap,P/E Ratio (TTM),52 Wk Change %
0,NVDA,NVIDIA Corporation,141.54,+1.13,+0.80%,203.386M,319.251M,3.472T,66.14,243.87%
1,TSLA,"Tesla, Inc.",269.19,+8.71,+3.34%,160.035M,80.132M,864.116B,73.55,36.40%
2,JOBY,"Joby Aviation, Inc.",5.16,-0.88,-14.57%,80.953M,9.143M,4.469B,-,0.19%
3,NIO,NIO Inc.,5.26,+0.28,+5.62%,65.584M,63.518M,11.081B,-,-30.05%
4,IBRX,"ImmunityBio, Inc.",5.32,+1.11,+26.37%,64.545M,3.938M,3.706B,-,72.73%
5,VALE,Vale S.A.,10.88,+0.39,+3.72%,59.907M,30.375M,46.447B,5.04,-19.53%
6,SOFI,"SoFi Technologies, Inc.",10.99,+0.06,+0.55%,57.497M,44.675M,11.715B,-,58.36%
7,DJT,Trump Media & Technology Group Corp.,38.95,+4.00,+11.44%,55.875M,18.256M,7.796B,-,157.27%
8,MARA,"MARA Holdings, Inc.",17.06,-1.16,-6.37%,55.351M,35.752M,5.321B,18.96,92.77%
9,INTC,Intel Corporation,22.68,+0.34,+1.52%,62.82M,82.316M,96.98B,94.50,-36.45%


In [108]:
df.to_csv('Most Active Stocks.csv')